In [1]:
import mysql.connector
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import tensorflow as tf

In [2]:
# Suppress TensorFlow warnings
tf.get_logger().setLevel('ERROR')

In [3]:
# Database connection details
db_config = {
    'host': 'localhost',          # Change if necessary
    'user': 'root',               # Your MySQL username
    'password': '',               # Your MySQL password
    'database': 'skin_cancer_prediction'  # Your database name
}

In [4]:
# Connect to the database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor(dictionary=True)

In [5]:
# Query to get the latest entry (or adjust based on your needs)
query = "SELECT * FROM predictions ORDER BY submission_date DESC LIMIT 1"
cursor.execute(query)
result = cursor.fetchone()

In [6]:
# Close the database connection
cursor.close()
conn.close()

In [7]:
# Extract data from the result
image_path = result['image_path']
learning_rate = result.get('learning_rate')
num_epochs = result.get('num_epochs')
batch_size = result.get('batch_size')

print(f"Retrieved data - Image Path: {image_path}, Learning Rate: {learning_rate}, Epochs: {num_epochs}, Batch Size: {batch_size}")

Retrieved data - Image Path: /Applications/XAMPP/xamppfiles/htdocs/cancer/uploads/ISIC_0029305.jpg, Learning Rate: 0.01, Epochs: 10, Batch Size: 32


In [8]:
# Load and preprocess the image
def load_and_preprocess_image(image_path):
    # Change target size to match the model's expected input shape
    img = image.load_img(image_path, target_size=(100, 100))  # Resize to 100x100 as expected by the model
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize the image if required by the model
    return img_array

In [9]:
# Check if the image file exists
if not os.path.exists(image_path):
    print(f"Error: The image path '{image_path}' does not exist.")
else:
    # Load the trained model
    model = load_model('model.h5')

    # Load and preprocess the image for prediction
    preprocessed_image = load_and_preprocess_image(image_path)

    # Ensure the shape is correct before prediction
    print(f"Preprocessed image shape: {preprocessed_image.shape}")

    # Make prediction
    prediction = model.predict(preprocessed_image)

    # Display the result (assuming binary classification; adjust as necessary)
    # For example, if your model predicts a value close to 0 -> 'Benign', close to 1 -> 'Malignant'
    threshold = 0.5
    result_label = 'Malignant' if prediction[0][0] > threshold else 'Benign'
    prediction_score = prediction[0][0]

    # Calculate and display the accuracy/confidence
    confidence = prediction_score if result_label == 'Malignant' else (1 - prediction_score)
    accuracy_percentage = confidence * 100

    print(f"The model predicts: {result_label} (Score: {prediction_score:.4f})")
    print(f"Prediction Confidence: {accuracy_percentage:.2f}%")

Preprocessed image shape: (1, 100, 100, 3)
1/1 [==============================] - 0s 123ms/step
The model predicts: Malignant (Score: 0.6142)
Prediction Confidence: 61.42%


2024-10-01 20:01:06.311423: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
